# Compute the atom centered Coulomb Matrix representation

To install QML library and compare rascal results with it
+ pip install qml ase

To install rascal:
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=ON ..
+ make -j 4
+ make install

In [10]:
import qml
from ase.io import read
from ase.visualize import view
import numpy as np
import sys

In [11]:
p = '/local/git/librascal/'
sys.path.insert(0,p+'build/')

In [12]:
from rascal.representation import SortedCoulombMatrix

In [13]:
# load a small subset of structures from QM9
frames = read(p+'tests/reference_data/dft-smiles_500.xyz',':')
cutoff = 3.
rep = SortedCoulombMatrix(cutoff)

In [14]:
# have a look at them
view(frames)

In [15]:
# Compute the sorted coulomb matrices for the list of structures
features = rep.transform(frames)

In [16]:
# extract the feature matrix to compare with 
test = features.get_feature_matrix().T

In [8]:
# To get the coulomb matrices from QML
def get_coulomb_ref(frame,size,cutoff):
    from qml.representations import generate_atomic_coulomb_matrix
    
    nuclear_charges = frame.get_atomic_numbers()
    coordinates = frame.get_positions()
    
    
    cm = generate_atomic_coulomb_matrix(nuclear_charges, coordinates, size = size, sorting = "distance",
    central_cutoff = cutoff, central_decay = -1, interaction_cutoff = 1e6, interaction_decay = -1,indices = None)
    
    return cm
def get_coulomb_refs(frames,size,cutoff):  
    cms = []
    for frame in frames:
        cms.append(get_coulomb_ref(frame,size,cutoff))
    return np.vstack(cms)

In [9]:
# Compare with reference
ref = get_coulomb_refs(frames,rep.size,cutoff)
np.allclose(test,ref)

True

In [11]:
from rascal.lib import sparsification

In [14]:
sparsification.fps?